In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import pickle
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt, resample
import copy
import time
import datetime
from multiprocessing.dummy import Pool as ThreadPool
from PreprocessFcns import *

%matplotlib inline

In [2]:
#  save corrected version of motor scores.xls 
# use updated 508/9 form instead
mot_scores = pd.read_excel('X:CIS-PD Study\\Scores\\reformatted_form509.xls')
# update score formatting
# fix error in original cols labels
mot_scores.columns.values[75] = 'alternating left hand movements tremor right upper limb'
mot_scores.columns.values[150] = 'sitting tremor right upper limb'
# remove words:(Qxx) and 'rating' from each column for readability
cols= mot_scores.columns
cols = cols.tolist()
colsnew = [x.split('(')[0] for x in cols]
colsnew = [x.strip() for x in colsnew] # remove whitespace
colsnew = [x.split('rating')[0] for x in colsnew]
colsnew = [x.strip() for x in colsnew]
colsnew = [x.lower() for x in colsnew] # make all lower case 
colsnew = [x.replace('\x97',' ') for x in colsnew]
colsnew = [x.replace('—',' ') for x in colsnew]    
# simplify notation
for i in range(len(colsnew)):
    x = colsnew[i]
    if x.find('finger to nose')>-1:
        colsnew[i] = x.replace(' hand','')
    if x.find('alternating')>-1:
        colsnew[i] = x.replace(' hand movements','')
c = dict(zip(cols,colsnew))
mot_scores = mot_scores.rename(index=str, columns=c)
# change 1 month label to 4 wks for proper sorting (to incorporate in xls file)
mot_scores.loc[mot_scores['visit']=='1 Month','visit']='4 Weeks'
# mot_scores.to_excel('Z:CIS-PD Study\Scores\MotorTasks.xls') #note that timestamps are not properly converted

In [3]:
subj_visit_list = pd.read_csv("X:\\CIS-PD Study\\subject visit list.csv")
subj_visit_list['FormDate'] = pd.to_datetime(subj_visit_list['FormDate'])

user_id_pairings = pd.read_csv("X:\\CIS-PD MUSC\\decoded_forms\\videoID.csv")

success_rates = []


In [4]:
def DataAggregator(subjDict,taskList_Abb,taskScores,sessionList,freq):
    
    Data = pd.DataFrame()
    
    numSamples = pd.DataFrame()
    
    s = 0
    for subj in list(subjDict.keys()):
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subjDict)))
        
        for t in range(len(taskList_Abb)):
            
            task = taskList_Abb[t]
            task_score = taskScores[t]
            
            for trial in range(len(sessionList)):
                
                visit = sessionList[trial]
                
                try:
                    data = pd.read_csv('X:CIS-PD Study\\MJFF Curation\\TaskAcc\\' + str(subj) + '_' + str(trial) + 
                                      '_' + task + '.csv',parse_dates=['timestamp'])[['timestamp','x','y','z']]
                
                except:
                    #print('No data found for %s trial %d'%(task,trial))
                    continue
                    
                side = subjDict[subj]
                
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                            task_score+ ' ' + 'bradykinesia ' + side + ' upper limb',
                                            task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score = subj_score.rename(index=str,columns={subj_score.columns[2]:'Bradykinesia',subj_score.columns[3]:'Tremor'})
                subj_score.index = range(len(subj_score))
                    
                data['timestamp'] = (data.timestamp.values - data.timestamp.values[0]).astype('timedelta64[ms]').astype(int)
                data = data.set_index('timestamp')
                
                data = HPfilter(data)
                
                clip_data = gen_clips_mc10(data,downsample=freq,basefreq=50)
                
                for c in range(len(clip_data['data'])):
                    N = pd.DataFrame()
                    N['Subject'] = subj
                    N['Task'] = task
                    N['Visit'] = sessionList[trial]
                    N['Samples'] = len(clip_data['data'][c])
                    numSamples = pd.concat([numSamples,N])
                
                feature_extraction_reduced(clip_data)
                
                if 'features' in clip_data.keys():
                    D = clip_data['features']
                    featcols = list(D.columns)
                    D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                    D['Tremor'] = subj_score['Tremor'][trial]
                    D['Visit'] = visit
                    D['Task'] = task
                    D['Subject'] = subj
                    Data = pd.concat([Data,D])
                    
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    
    #if freq==50:
        #numSamples.to_csv('Z:CIS-PD Study\\Watch Sample Lengths.csv')
    
    return Data
    
    #Data.to_csv('Z:CIS-PD Study\\Smartwatch Data 50Hz.csv')

In [5]:
# get the 4 digit user_id from the corresponding 6 digit user_id
def user_id_6_to_4(user_id_6):
    for index, row in user_id_pairings.iterrows():
        if (not np.isnan(row['Subj ID Athena'])):
            if (int(row['Subj ID Athena'])==int(user_id_6)):
                return int(row['SubjectCode'])
    return None

In [6]:
def HPfilter(rawdata,cutoff=0.75,ftype='highpass'):
#highpass (or lowpass) filter data. HP to remove gravity (offset - limb orientation) from accelerometer data from each visit (trial)
#input: Activity dictionary, cutoff freq [Hz], task, sensor location and type of filter (highpass or lowpass).

    if rawdata.empty is False: #skip if no data for current sensor
        idx = rawdata.index
        idx = idx-idx[0]
        rawdata.index = idx
        x = rawdata.values # modify to ignore other columns
        Fs = np.mean(1/(np.diff(rawdata.index)/1000)) #sampling rate

        #filter design
        cutoff_norm = cutoff/(0.5*Fs)

        b,a = butter(4,cutoff_norm,btype=ftype,analog=False)
        #filter data
        xfilt = filtfilt(b,a,x,axis=0)
        rawdatafilt = pd.DataFrame(data=xfilt,index=rawdata.index,columns=rawdata.columns)
        return rawdatafilt

In [7]:
def HomeDataAggregator(file):
    t1 = time.time()
    Data = pd.DataFrame()
    print(file)
    
    features_list = ['RMSX','RMSY','RMSZ','rangeX','rangeY','rangeZ','meanX','meanY','meanZ','varX','varY','varZ',
                    'skewX','skewY','skewZ','kurtX','kurtY','kurtZ','xcor_peakXY','xcorr_peakXZ','xcorr_peakYZ',
                    'xcorr_lagXY','xcorr_lagXZ','xcorr_lagYZ','Dom_freq','Pdom_rel','PSD_mean','PSD_std','PSD_skew',
                    'PSD_kur','jerk_mean','jerk_std','jerk_skew','jerk_kur']
        
    # get acc data
    try:
        # change if we want to use Activity Level and Tremor Score
        data = pd.read_pickle('X:\\CIS-PD Study\\MJFF Curation\\combined pre-visit data\\' + 
                               file)[['user_id','timestamp','Gait','x','y','z']]  
    except:
        print('No data found for %s trial %d'%(task,trial))
        return
        
    # organize data and make 5 second clips
    data = data.sort_values(by = 'timestamp', axis = 0)
    
    data['timestamp2'] = [(tm - datetime.timedelta(minutes=0,
                                                   seconds=tm.second % 5,
                                                   microseconds=tm.microsecond)) 
                          for tm in data.timestamp]
    
    data['timestamp'] = (data.timestamp.values - data.timestamp.values[0]).astype('timedelta64[ms]').astype(int)
    data = data.set_index('timestamp')
    data.loc[:,['x', 'y', 'z']] = HPfilter(data[['x', 'y', 'z']])
                
    # "clip" the data into 5 second chunks    
    five_sec_intervals = data.timestamp2.unique()
        
    # calculate features
    F=[]
    num_empty = 0
    times = []
    for t in five_sec_intervals:
        clip = data.loc[(data.timestamp2 == t)] # & (data.Gait == 1)] # take only walking data
        if (clip.empty or (len(clip.timestamp2) < 200)):
            num_empty += 1
        else:
            F.append(reduced_feature_extraction_from_1_clip(clip[['x', 'y', 'z']]))
            times.append(t)

    success_info = [file, len(five_sec_intervals), (len(five_sec_intervals)-num_empty)]
    df = pd.DataFrame(data = [success_info], columns = ['file', 'num expected clips', 'num actual clips'])
    if (os.path.isfile("X:\\CIS-PD Study\\Home WACC features\\success info.csv")):
            dfo = pd.read_csv("X:\\CIS-PD Study\\Home WACC features\\success info.csv", index_col = 0)
            df = pd.concat([dfo, df])
    df.to_csv("X:\\CIS-PD Study\\Home WACC features\\success info.csv")
            
    # create features dataframe
    D = pd.DataFrame(data=F,columns=features_list,dtype='float32')    
    featcols = list(D.columns)
    D['Subject'] = data.loc[0, 'user_id']
    D['timestamp'] = times #['timestamp2']
    Data = pd.concat([Data,D])   
    cols = ['Subject','timestamp'] + featcols
    Data = Data[cols]
    elapsed_time = ((time.time() - t1)/60.0).__str__()
    print(elapsed_time + " mins\n")
    Data.to_pickle('X:\CIS-PD Study\Home WACC features\\features ' + file)

In [8]:
def mapping_func(t, data):
    clip = data.loc[(data.timestamp2 == t)] # & (data.Gait == 1)] # take only walking data
    if not((clip.empty or (len(clip.index) < 200))):
        return list(reduced_feature_extraction_from_1_clip(clip[['x', 'y', 'z']])).append(t)


In [9]:
def HomeDataAggregatorThreading(file):
    t1 = time.time()
    Data = pd.DataFrame()
    print(file)
    
    features_list = ['RMSX','RMSY','RMSZ','rangeX','rangeY','rangeZ','meanX','meanY','meanZ','varX','varY','varZ',
                    'skewX','skewY','skewZ','kurtX','kurtY','kurtZ','xcor_peakXY','xcorr_peakXZ','xcorr_peakYZ',
                    'xcorr_lagXY','xcorr_lagXZ','xcorr_lagYZ','Dom_freq','Pdom_rel','PSD_mean','PSD_std','PSD_skew',
                    'PSD_kur','jerk_mean','jerk_std','jerk_skew','jerk_kur', 'timestamp']
        
    # get acc data
    try:
        # change if we want to use Activity Level and Tremor Score
        data = pd.read_pickle('X:\\CIS-PD Study\\MJFF Curation\\combined pre-visit data\\' + 
                               file)[['user_id','timestamp','Gait','x','y','z']]  
    except:
        print('No data found for %s trial %d'%(task,trial))
        return
        
    # organize data and make 5 second clips
    data = data.sort_values(by = 'timestamp', axis = 0)
    
    data['timestamp2'] = [(tm - datetime.timedelta(minutes=0,
                                                   seconds=tm.second % 5,
                                                   microseconds=tm.microsecond)) 
                          for tm in data.timestamp]
    
    data['timestamp'] = (data.timestamp.values - data.timestamp.values[0]).astype('timedelta64[ms]').astype(int)
    data = data.set_index('timestamp')
    data.loc[:,['x', 'y', 'z']] = HPfilter(data[['x', 'y', 'z']])
                
    # "clip" the data into 5 second chunks    
    five_sec_intervals = data.timestamp2.unique()
        
    # calculate features
    
    
    pool = ThreadPool(4)
    # does type of results need to change?
    results = pool.map(lambda t : mapping_func(t, data), five_sec_intervals)
    pool.close()
    pool.join()

    print("\n\n")
    print(results)
    results = list(results)
    print('\n')
    print (results)
    results = list(filter(lambda r: r is not None, results))

    success_info = [file, len(five_sec_intervals), len(results)]
    df = pd.DataFrame(data = [success_info], columns = ['file', 'num expected clips', 'num actual clips'])
    if (os.path.isfile("X:\\CIS-PD Study\\Home WACC features\\success info.csv")):
            dfo = pd.read_csv("X:\\CIS-PD Study\\Home WACC features\\success info.csv", index_col = 0)
            df = pd.concat([dfo, df])
    df.to_csv("X:\\CIS-PD Study\\Home WACC features\\success info.csv")
            
    # create features dataframe
    D = pd.DataFrame(data=results,columns=features_list,dtype='float32')    
    featcols = list(D.columns)
    D['Subject'] = data.loc[0, 'user_id']
    Data = pd.concat([Data,D])   
    cols = ['Subject'] + featcols
    Data = Data[cols]
    elapsed_time = ((time.time() - t1)/60.0).__str__()
    print(elapsed_time + " mins\n")
    Data.to_pickle('X:\CIS-PD Study\Home WACC features\\features ' + file)

In [10]:
#subjDict = {1004:'right',1016:'left',1018:'left',1019:'left',1020:'right',1024:'left',1029:'left',1030:'left',1032:'left',
#            1038:'left',1044:'right',1046:'right',1049:'left',1051:'left'}
subjDict = {1004:'right'}
#1047

taskList = ['Standing', 'Walking', 'Walking while counting', 'Finger to nose--right hand', 
            'Finger to nose--left hand', 'Alternating right hand movements', 'Alternating left hand movements', 
            'Sit to stand', 'Drawing on a paper', 'Typing on a computer keyboard', 'Assembling nuts and bolts', 
            'Taking a glass of water and drinking', 'Organizing sheets in a folder', 'Folding towels', 'Sitting']

taskScores = ['standing','walking','walking while counting','finger to nose right','finger to nose left',
                   'alternating right','alternating left','sit to stand','drawing on a paper',
                   'typing on a computer keyboard','assembling nuts and bolts','taking a glass of water and drinking',
                   'organizing sheets in a folder','folding towels','sitting']

taskList_Abb = ['Stndg', 'Wlkg', 'WlkgCnt', 'FtnR', 'FtnL', 'RamR', 'RamL', 'SitStand', 'Drwg', 'Typg', 'NtsBts',
                'Drnkg', 'Sheets', 'Fldg', 'Sitng']

sessionList = ['2 Weeks: Time 0', '2 Weeks: Time 30', '2 Weeks: Time 60', '2 Weeks: Time 90', '2 Weeks: Time 120', 
               '2 Weeks: Time 150', '4 Weeks']

features_list = ['RMSX','RMSY','RMSZ','rangeX','rangeY','rangeZ','meanX','meanY','meanZ','varX','varY','varZ',
                    'skewX','skewY','skewZ','kurtX','kurtY','kurtZ','xcor_peakXY','xcorr_peakXZ','xcorr_peakYZ',
                    'xcorr_lagXY','xcorr_lagXZ','xcorr_lagYZ','Dom_freq','Pdom_rel','PSD_mean','PSD_std','PSD_skew',
                    'PSD_kur','jerk_mean','jerk_std','jerk_skew','jerk_kur']

In [11]:
#for freq in [50,45,40,35,30,25,20,15,10,5]:
#    print(freq)
#    t1 = time.time()
#    Data = DataAggregator(subjDict,taskList_Abb,taskScores,sessionList,freq)
#    #Data.to_csv('X:\CIS-PD Study\Clinic WACC features\\Watch Data ' + str(freq) +  'Hz.csv')
#    t2 = (time.time() - t1)/60.0
#    print(t2.__str__() + " mins")

In [12]:
# All Files:
#files = os.listdir("X:\\CIS-PD Study\\MJFF Curation\\combined pre-visit data")

# Luca's Files:
# files = ['142605 2017-08-16.pkl', '142557 2017-09-21.pkl', '142559 2017-07-06.pkl', '142617 2017-07-17.pkl', '142582 2017-07-27.pkl', '142577 2017-11-30.pkl', '142604 2017-10-02.pkl', '142604 2017-07-21.pkl', '142568 2017-09-18.pkl', '142615 2017-07-21.pkl', '142570 2017-07-13.pkl', '142623 2017-10-12.pkl', '142559 2017-07-20.pkl', '142602 2017-12-12.pkl', '142592 2017-10-03.pkl', '142561 2017-07-03.pkl', '142584 2018-02-21.pkl', '142562 2017-07-07.pkl', '142622 2017-08-03.pkl', '142606 2017-10-13.pkl', '142604 2017-08-14.pkl', '142568 2018-02-28.pkl', '142583 2017-10-13.pkl', '142618 2017-07-31.pkl', '142583 2017-08-25.pkl', '142603 2017-08-07.pkl', '142615 2017-10-05.pkl', '142595 2017-10-12.pkl', '142593 2018-01-29.pkl', '142623 2017-08-17.pkl', '142601 2017-07-13.pkl', '142570 2017-08-03.pkl', '142568 2017-09-06.pkl', '142602 2017-08-03.pkl', '142618 2017-10-30.pkl', '142616 2018-02-05.pkl', '142584 2017-11-29.pkl', '142598 2017-07-24.pkl', '142608 2017-09-18.pkl']

# Nick's Files:
# files = ['142598 2017-07-10.pkl', '142617 2017-08-01.pkl', '142603 2017-07-31.pkl', '142605 2017-08-02.pkl', '142618 2018-01-30.pkl', '142592 2017-07-13.pkl', '142558 2017-06-29.pkl', '142620 2017-07-21.pkl', '142593 2017-07-20.pkl', '142617 2017-10-03.pkl', '142594 2017-08-10.pkl', '142592 2017-07-27.pkl', '142578 2018-02-19.pkl', '142621 2017-07-21.pkl', '142612 2017-09-06.pkl', '142609 2017-08-11.pkl', '142612 2017-08-15.pkl', '142619 2017-08-10.pkl', '142608 2017-09-11.pkl', '142558 2017-07-14.pkl', '142585 2017-10-04.pkl', '142595 2018-01-10.pkl', '142557 2017-06-29.pkl', '142622 2017-10-12.pkl', '142601 2017-07-26.pkl', '142575 2017-08-29.pkl', '142581 2017-07-26.pkl', '142566 2017-07-26.pkl', '142608 2017-11-09.pkl', '142602 2017-09-20.pkl', '142606 2017-12-29.pkl', '142567 2017-07-31.pkl', '142620 2017-08-10.pkl', '142622 2017-08-17.pkl', '142583 2017-08-09.pkl', '142602 2017-07-13.pkl', '142566 2017-08-09.pkl', '142580 2017-08-01.pkl']

# Julianne's Files:
# files = ['142618 2017-08-14.pkl', '142557 2017-07-13.pkl', '142585 2018-02-02.pkl', '142581 2017-08-14.pkl', '142577 2017-08-03.pkl', '142619 2018-01-22.pkl', '142584 2017-11-01.pkl', '142600 2017-07-11.pkl', '142620 2017-10-30.pkl', '142593 2017-08-03.pkl', '142621 2017-10-23.pkl', '142609 2017-09-01.pkl', '142578 2017-09-05.pkl', '142616 2017-07-19.pkl', '142593 2017-11-06.pkl', '142600 2017-07-25.pkl', '142606 2017-08-16.pkl', '142616 2017-08-01.pkl', '142615 2017-07-07.pkl', '142595 2017-08-22.pkl', '142615 2018-01-22.pkl', '142598 2017-09-13.pkl', '142557 2017-12-11.pkl', '142601 2017-09-20.pkl', '142582 2017-08-09.pkl', '142594 2017-07-19.pkl', '142577 2017-10-11.pkl', '142579 2017-07-18.pkl', '142619 2017-07-21.pkl', '142578 2017-09-18.pkl', '142605 2017-10-11.pkl', '142575 2017-08-16.pkl', '142580 2017-07-14.pkl', '142623 2017-08-03.pkl', '142563 2017-07-18.pkl', '142604 2017-11-27.pkl', '142584 2017-09-13.pkl', '142616 2017-10-03.pkl', '142560 2017-07-03.pkl']

for file in files:
    Data = HomeDataAggregator(file)

142567 2018-01-11.pkl
14.082773649692536 mins
142558 2017-09-21.pkl
9.78990361293157 mins
142579 2017-07-07.pkl
8.254690742492675 mins
142621 2017-08-14.pkl
22.94182907342911 mins
142585 2017-09-20.pkl
20.22361896832784 mins
142580 2018-01-10.pkl
14.430702904860178 mins
142603 2017-10-13.pkl
12.601396624247233 mins
142583 2018-01-30.pkl
20.280219427744548 mins
142618 2017-08-14.pkl


KeyboardInterrupt: 

In [ ]:
#pool = ThreadPool(2)
#pool.map(HomeDataAggregator, files)
#pool.close()
#pool.join()